In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [3]:
train_data_dir = 'dataset/train'
validation_data_dir = 'dataset/validation'
img_width, img_height = 224, 224
num_classes = 2 # "Hasarlı" ve "Hasarsız" sınıfları için


In [4]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical')

Found 1840 images belonging to 2 classes.
Found 460 images belonging to 2 classes.


In [5]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical'
)


Found 1840 images belonging to 2 classes.
Found 460 images belonging to 2 classes.


In [6]:
from tensorflow.keras.applications import ResNet50
from keras.models import Sequential

# model tanımlama
model = Sequential()
model.add(ResNet50(include_top=False, pooling='avg', weights='imagenet'))
model.add(Dense(2, activation='softmax'))

# compile model
model.layers[0].trainable = False
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [7]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=x)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [8]:
history = model.fit(
    train_generator,
    steps_per_epoch=28, #train_data_size/batch
    epochs=100,
    validation_data=validation_generator,
    validation_steps=7, #validation_data_size/batch
)
model.save('model.h5')


Epoch 1/10
28/28 [==============================] - 73s 2s/step - loss: 10.2467 - accuracy: 0.5250 - val_loss: 4.4822 - val_accuracy: 0.5089
Epoch 2/10
28/28 [==============================] - 68s 2s/step - loss: 4.2244 - accuracy: 0.4777 - val_loss: 0.7169 - val_accuracy: 0.5268
Epoch 3/10
28/28 [==============================] - 66s 2s/step - loss: 1.2307 - accuracy: 0.5156 - val_loss: 0.6816 - val_accuracy: 0.5357
Epoch 4/10
28/28 [==============================] - 66s 2s/step - loss: 0.7055 - accuracy: 0.4900 - val_loss: 0.6887 - val_accuracy: 0.5223
Epoch 5/10
28/28 [==============================] - 65s 2s/step - loss: 0.7158 - accuracy: 0.5045 - val_loss: 0.6865 - val_accuracy: 0.5089
Epoch 6/10
28/28 [==============================] - 66s 2s/step - loss: 0.7044 - accuracy: 0.5089 - val_loss: 0.6918 - val_accuracy: 0.5089
Epoch 7/10
28/28 [==============================] - 67s 2s/step - loss: 0.7029 - accuracy: 0.5234 - val_loss: 0.6811 - val_accuracy: 0.5089
Epoch 8/10
28/28 [=

In [9]:
from keras.models import load_model

model = load_model('model.h5')

In [10]:
from tensorflow.keras.preprocessing import image
import numpy as np

img_path = 'dataset/test/28.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

In [11]:
preds = model.predict(x)
if preds[0][0] > preds[0][1]:
    print("Hasarlı")
else:
    print("Değil")


1/1 [==============================] - 1s 679ms/step
Hasarlı
